In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.transforms import Affine2D
from matplotlib.projections import PolarAxes
import mpl_toolkits.axisartist.floating_axes as floating_axes
import mpl_toolkits.axisartist.angle_helper as angle_helper
from mpl_toolkits.axisartist.grid_finder import FixedLocator, MaxNLocator, DictFormatter

from astropy.table import Table
from astropy.io import fits
from astropy.cosmology import FlatLambdaCDM, z_at_value

from scipy.spatial import cKDTree, ConvexHull

from sympy import solve_poly_system, im
from sympy.abc import x,y

matplotlib.rcParams.update({'font.size': 38})

In [ ]:
#galaxy data file
gdata = fits.open("ALL.fits")
#VoidFinder maximal sphere output
vfdata = Table.read("DR7_comoving_maximal.txt",format='ascii.commented_header')
#VoidFinder hole output
vfdata2 = Table.read("DR7_comoving_holes.txt",format='ascii.commented_header')
#Vsquared triangle output
tridata = Table.read("DR7_triangles.dat",format='ascii.commented_header')
gzdata = Table.read("DR7_galzones.dat",format='ascii.commented_header')
zvdata = Table.read("DR7_zonevoids.dat",format='ascii.commented_header')

In [ ]:
D2R = np.pi/180.

def toSky(cs):
    c1  = cs.T[0]
    c2  = cs.T[1]
    c3  = cs.T[2]
    r   = np.sqrt(c1**2.+c2**2.+c3**2.)
    dec = np.arcsin(c3/r)/D2R
    ra  = (np.arccos(c1/np.sqrt(c1**2.+c2**2.))*np.sign(c2)/D2R)%360
    return r,ra,dec

def toCoord(r,ra,dec):
    c1 = r*np.cos(ra*D2R)*np.cos(dec*D2R)
    c2 = r*np.sin(ra*D2R)*np.cos(dec*D2R)
    c3 = r*np.sin(dec*D2R)
    return c1,c2,c3

In [ ]:
gr   = gdata[1].data['Rgal']
gra  = gdata[1].data['ra']
gdec = gdata[1].data['dec']

gx,gy,gz = toCoord(gr,gra,gdec)
kdt = cKDTree(np.array([gx,gy,gz]).T)

g_z = gzdata['zone']
z_v = zvdata['void0']

vfx = vfdata['x']
vfy = vfdata['y']
vfz = vfdata['z']
vfr,vfra,vfdec = toSky(np.array([vfx,vfy,vfz]).T)
vfrad = vfdata['radius']
vfc  = matplotlib.cm.nipy_spectral(np.linspace(0,1,len(vfr)))
vfcc = np.random.choice(range(len(vfc)),len(vfc),replace=False)

vflag = vfdata2['flag']
vfx2 = vfdata2['x']
vfy2 = vfdata2['y']
vfz2 = vfdata2['z']
vfr1,vfra1,vfdec1 = toSky(np.array([vfx2,vfy2,vfz2]).T)
vfrad1 = vfdata2['radius']
vfx4   = [vfx2[vflag==vfl] for vfl in np.unique(vflag)]
vfy4   = [vfy2[vflag==vfl] for vfl in np.unique(vflag)]
vfz4   = [vfz2[vflag==vfl] for vfl in np.unique(vflag)]
vfr2   = [vfr1[vflag==vfl] for vfl in np.unique(vflag)]
vfra2  = [vfra1[vflag==vfl] for vfl in np.unique(vflag)]
vfdec2 = [vfdec1[vflag==vfl] for vfl in np.unique(vflag)]
vfrad2 = [vfrad1[vflag==vfl] for vfl in np.unique(vflag)]

gflag_vf = np.zeros(len(gx),dtype=bool)
gflag_v2 = np.zeros(len(gx),dtype=bool)

for vfl in np.unique(vflag):
    vfx3 = vfx2[vflag==vfl]
    vfy3 = vfy2[vflag==vfl]
    vfz3 = vfz2[vflag==vfl]
    vfrad3 = vfrad1[vflag==vfl]
    for i in range(len(vfx3)):
        galinds = kdt.query_ball_point([vfx3[i],vfy3[i],vfz3[i]],vfrad3[i])
        gflag_vf[galinds] = True

for z in range(len(z_v)):
    if z_v[z] > -1:
        gflag_v2[g_z==z] = True

wflag_vf = (1-gflag_vf).astype(bool)
wflag_v2 = (1-gflag_v2).astype(bool)

p1_r,p1_ra,p1_dec = toSky(np.array([tridata['p1_x'],tridata['p1_y'],tridata['p1_z']]).T)
p2_r,p2_ra,p2_dec = toSky(np.array([tridata['p2_x'],tridata['p2_y'],tridata['p2_z']]).T)
p3_r,p3_ra,p3_dec = toSky(np.array([tridata['p3_x'],tridata['p3_y'],tridata['p3_z']]).T)
p1_x = tridata['p1_x']
p1_y = tridata['p1_y']
p1_z = tridata['p1_z']
p2_x = tridata['p2_x']
p2_y = tridata['p2_y']
p2_z = tridata['p2_z']
p3_x = tridata['p3_x']
p3_y = tridata['p3_y']
p3_z = tridata['p3_z']
trivids = np.array(tridata['void_id'])
v2c  = matplotlib.cm.nipy_spectral(np.linspace(0,1,np.amax(trivids)+1))
v2cc = np.random.choice(range(len(v2c)),len(v2c),replace=False)

In [ ]:
#calculate radii of maximal sphere-slice intersections
def cint(dec):
    cr = []
    for i in range(len(vfr)):
        dtd = np.abs(vfr[i]*np.sin((vfdec[i]-dec)*D2R))
        if dtd>vfrad[i]:
            cr.append(0.)
        else:
            cr.append(np.sqrt(vfrad[i]**2.-dtd**2.))
    return cr

#calculate radii of hole-slice intersections
def cint2(dec):
    cr = []
    for i in range(len(vfr2)):
        cr.append([])
        for j in range(len(vfr2[i])):
            dtd = np.abs(vfr2[i][j]*np.sin((vfdec2[i][j]-dec)*D2R))
            if dtd>vfrad2[i][j]:
                cr[i].append(0.)
            else:
                cr[i].append(np.sqrt(vfrad2[i][j]**2.-dtd**2.))
    return cr

def isin(p,ps,ch,chavg,chrad):
    if np.sum((p-chavg)**2.)<chrad:
        return True
    nc = 1
    for smp in ch.simplices:
        ps2 = ps[smp]
        if ps2[0][0]<p[0]:
            continue
        elif ps2[1][0]<p[0]:
            continue
        elif (ps2[0][1]-p[1])*(ps2[1][1]-p[1])<0:
            nc = nc+1
        elif (ps2[0][1]-p[1])*(ps2[1][1]-p[1])==0:
            nc = nc+0.5
    return nc%2==0

def isin2(p,ps):
    nc = 1
    for i in range(len(ps)-1):
        if p1[0]<p[0] and p2[0]<p[0]:
            continue
        elif (p1[1]-p[1])*(p2[1]-p[1])>0:
            continue
        elif p1[0]>p[0] and p2[0]>p[0]:
            nc = nc+1
        elif ((p2[1]-p1[1])/(p2[0]-p1[0]))*((p1[1]-p[1])-((p2[1]-p1[1])/(p2[0]-p1[0]))*(p1[0]-p[0]))<1:
            nc = nc+1
    return nc%2==0

#calculate coordinates of triangle-slice intersections
def trint(dec):
    decsum = np.array([(p1_dec>dec).astype(int),(p2_dec>dec).astype(int),(p3_dec>dec).astype(int)]).T
    intr  = [[] for _ in range(np.amax(trivids)+1)]
    intra = [[] for _ in range(np.amax(trivids)+1)]
    for i in range(len(trivids)):
        if np.sum(decsum[i])==0:
            continue
        if np.sum(decsum[i])==3:
            continue
        cv = trivids[i]
        if np.sum(decsum[i])==1:
            if decsum[i][0]==1:
                intr[cv].append((p1_r[i]+p2_r[i])/2.)
                intr[cv].append((p1_r[i]+p3_r[i])/2.)
                intra[cv].append((p1_ra[i]+p2_ra[i])/2.)
                intra[cv].append((p1_ra[i]+p3_ra[i])/2.)
            elif decsum[i][1]==1:
                intr[cv].append((p2_r[i]+p1_r[i])/2.)
                intr[cv].append((p2_r[i]+p3_r[i])/2.)
                intra[cv].append((p2_ra[i]+p1_ra[i])/2.)
                intra[cv].append((p2_ra[i]+p3_ra[i])/2.)
            elif decsum[i][2]==1:
                intr[cv].append((p3_r[i]+p1_r[i])/2.)
                intr[cv].append((p3_r[i]+p2_r[i])/2.)
                intra[cv].append((p3_ra[i]+p1_ra[i])/2.)
                intra[cv].append((p3_ra[i]+p2_ra[i])/2.)
        elif np.sum(decsum[i])==2:
            if decsum[i][0]==0:
                intr[cv].append((p1_r[i]+p2_r[i])/2.)
                intr[cv].append((p1_r[i]+p3_r[i])/2.)
                intra[cv].append((p1_ra[i]+p2_ra[i])/2.)
                intra[cv].append((p1_ra[i]+p3_ra[i])/2.)
            elif decsum[i][1]==0:
                intr[cv].append((p2_r[i]+p1_r[i])/2.)
                intr[cv].append((p2_r[i]+p3_r[i])/2.)
                intra[cv].append((p2_ra[i]+p1_ra[i])/2.)
                intra[cv].append((p2_ra[i]+p3_ra[i])/2.)
            elif decsum[i][2]==0:
                intr[cv].append((p3_r[i]+p1_r[i])/2.)
                intr[cv].append((p3_r[i]+p2_r[i])/2.)
                intra[cv].append((p3_ra[i]+p1_ra[i])/2.)
                intra[cv].append((p3_ra[i]+p2_ra[i])/2.)
    return intr,intra

def getinx(xx,aa,yy,bb,zz,cc,dd):
    negb = -1.*aa*xx-bb*yy+cc*dd*dd*zz
    sqto = 0.5*np.sqrt((2.*aa*xx+2.*bb*yy-2.*cc*dd*dd*zz)**2.-4.*(aa**2.+bb**2.-cc*cc*dd*dd)*(xx**2.+yy**2.-zz*zz*dd*dd))
    twa = aa**2.+bb**2.-cc*cc*dd*dd
    tt = (negb+sqto)/twa
    if tt>0 and tt<1:
        tt = tt
    else:
        tt = (negb-sqto)/twa
    return xx+aa*tt,yy+bb*tt,zz+cc*tt

def trint2(dec):
    decsum = np.array([(p1_dec>dec).astype(int),(p2_dec>dec).astype(int),(p3_dec>dec).astype(int)]).T
    intr  = [[] for _ in range(np.amax(trivids)+1)]
    intra = [[] for _ in range(np.amax(trivids)+1)]
    for i in range(len(trivids)):
        if np.sum(decsum[i])==0:
            continue
        if np.sum(decsum[i])==3:
            continue
        cv = trivids[i]
        if np.sum(decsum[i])==1:
            if decsum[i][0]==1:
                sss = getinx(p1_x[i],p2_x[i]-p1_x[i],p1_y[i],p2_y[i]-p1_y[i],p1_z[i],p2_z[i]-p1_z[i],1./np.tan(dec*D2R))
                sst = getinx(p1_x[i],p3_x[i]-p1_x[i],p1_y[i],p3_y[i]-p1_y[i],p1_z[i],p3_z[i]-p1_z[i],1./np.tan(dec*D2R))
            elif decsum[i][1]==1:
                sss = getinx(p1_x[i],p2_x[i]-p1_x[i],p1_y[i],p2_y[i]-p1_y[i],p1_z[i],p2_z[i]-p1_z[i],1./np.tan(dec*D2R))
                sst = getinx(p3_x[i],p2_x[i]-p3_x[i],p3_y[i],p2_y[i]-p3_y[i],p3_z[i],p2_z[i]-p3_z[i],1./np.tan(dec*D2R))
            elif decsum[i][2]==1:
                sss = getinx(p1_x[i],p3_x[i]-p1_x[i],p1_y[i],p3_y[i]-p1_y[i],p1_z[i],p3_z[i]-p1_z[i],1./np.tan(dec*D2R))
                sst = getinx(p3_x[i],p2_x[i]-p3_x[i],p3_y[i],p2_y[i]-p3_y[i],p3_z[i],p2_z[i]-p3_z[i],1./np.tan(dec*D2R))
        elif np.sum(decsum[i])==2:
            if decsum[i][0]==0:
                sss = getinx(p1_x[i],p2_x[i]-p1_x[i],p1_y[i],p2_y[i]-p1_y[i],p1_z[i],p2_z[i]-p1_z[i],1./np.tan(dec*D2R))
                sst = getinx(p1_x[i],p3_x[i]-p1_x[i],p1_y[i],p3_y[i]-p1_y[i],p1_z[i],p3_z[i]-p1_z[i],1./np.tan(dec*D2R))
            elif decsum[i][1]==0:
                sss = getinx(p1_x[i],p2_x[i]-p1_x[i],p1_y[i],p2_y[i]-p1_y[i],p1_z[i],p2_z[i]-p1_z[i],1./np.tan(dec*D2R))
                sst = getinx(p3_x[i],p2_x[i]-p3_x[i],p3_y[i],p2_y[i]-p3_y[i],p3_z[i],p2_z[i]-p3_z[i],1./np.tan(dec*D2R))
            elif decsum[i][2]==0:
                sss = getinx(p1_x[i],p3_x[i]-p1_x[i],p1_y[i],p3_y[i]-p1_y[i],p1_z[i],p3_z[i]-p1_z[i],1./np.tan(dec*D2R))
                sst = getinx(p3_x[i],p2_x[i]-p3_x[i],p3_y[i],p2_y[i]-p3_y[i],p3_z[i],p2_z[i]-p3_z[i],1./np.tan(dec*D2R))
        intr[cv].append(np.sqrt(np.sum(np.array(sss)**2.)))
        intr[cv].append(np.sqrt(np.sum(np.array(sst)**2.)))
        intra[cv].append((np.arccos(sss[0]/np.sqrt(sss[0]**2.+sss[1]**2.))*np.sign(sss[1])/D2R)%360)
        intra[cv].append((np.arccos(sst[0]/np.sqrt(sst[0]**2.+sst[1]**2.))*np.sign(sst[1])/D2R)%360)
    return intr,intra

In [ ]:
#convert a circle's coordinates to ordered boundary
def gcp(cc1,cc2,crad,npt):
    ccx = cc1*np.cos(cc2*D2R)
    ccy = cc1*np.sin(cc2*D2R)
    Cx = np.linspace(0.,2*np.pi,npt)
    Cy = np.linspace(0.,2*np.pi,npt)
    Cx = np.cos(Cx)*crad+ccx
    Cy = np.sin(Cy)*crad+ccy
    C1 = np.sqrt(Cx**2.+Cy**2.)
    C2 = (np.sign(Cy)*np.arccos(Cx/C1)+np.pi*(1.-np.sign(Cy)))/D2R
    return C1,C2

#convert circles' coordinates to ordered boundary
def gcp2(cc1,cc2,crad,npt,chkdpth):
    ccx = cc1*np.cos(cc2*D2R)
    ccy = cc1*np.sin(cc2*D2R)
    Cx = [np.linspace(0.,2*np.pi,int(npt*crad[k]/10)) for k in range(len(ccx))]
    Cy = [np.linspace(0.,2*np.pi,int(npt*crad[k]/10)) for k in range(len(ccx))]
    Cx = [np.cos(Cx[k])*crad[k]+ccx[k] for k in range(len(ccx))]
    Cy = [np.sin(Cy[k])*crad[k]+ccy[k] for k in range(len(ccx))]
    for i in range(len(ccx)):
        for j in range(len(ccx)):
            if i==j:
                continue
            cut = (Cx[j]-ccx[i])**2.+(Cy[j]-ccy[i])**2.>crad[i]**2.
            Cx[j] = Cx[j][cut]
            Cy[j] = Cy[j][cut]
    Cp = []
    for i in range(len(ccx)):
        Cp.extend(np.array([Cx[i],Cy[i]]).T.tolist())
    Cp = np.array(Cp)
    kdt = cKDTree(Cp)
    Cpi = [0]
    while len(Cpi)<len(Cp):
        if len(Cpi)==1:
            nid = kdt.query(Cp[Cpi[-1]],2)[1][1]
        else:
            nids = kdt.query(Cp[Cpi[-1]],chkdpth+1)[1][1:]
            for k in range(chkdpth):
                if nids[k] not in Cpi[(-1*(chkdpth+1)):-1]:
                    nid = nids[k]
                    break
            nids = kdt.query(Cp[Cpi[-1]],7)[1][1:]
        Cpi.append(nid)
    #Cpi.append(0)
    C1 = np.sqrt(Cp[Cpi].T[0]**2.+Cp[Cpi].T[1]**2.)
    C2 = (np.sign(Cp[Cpi].T[1])*np.arccos(Cp[Cpi].T[0]/C1)+np.pi*(1.-np.sign(Cp[Cpi].T[1])))/D2R
    return C1,C2

def gcp3(sx,sy,sz,sr,srad,sdec,dec,npt,chkdpth):
    Cx  = []
    Cy  = []
    Cz  = []
    Cr  = []
    Cra = []
    Cs  = []
    Cs2 = []
    Ch  = []
    Chavg = []
    Chrad = []
    print(len(sx))
    for i in range(len(sx)):
        print("|",end='',flush=True)
        dtd = np.abs(sr[i]*np.sin((sdec[i]-dec)*D2R))
        if dtd>srad[i]:
            continue
        else:
            frad = np.sqrt(srad[i]**2.-dtd**2.)
            Cx.append([])
            Cy.append([])
        A = srad[i]**2.
        B = (1.-np.sin(dec*D2R))*2.*sz[i]*srad[i]
        C = -1.*np.sin(dec*D2R)*2.*srad[i]*(sx[i]*np.cos(np.arange(int(npt*frad/20.))*20.*np.pi/(npt*frad))+sy[i]*np.sin(np.arange(int(npt*frad/20.))*20.*np.pi/(npt*frad)))
        D = (sz[i]**2.)-np.sin(dec*D2R)*(sx[i]**2+sy[i]**2.+sz[i]**2.+srad[i]**2.)
        adjstr = 0
        for j in range(len(C)):
            if C[j] != 0:
                try:
                    print(A)
                    print(B)
                    print(C[j])
                    print(D)
                    sps = solve_poly_system([A*(x**2)+B*x+C[j]*y+D,x**2+y**2-1],x,y)
                    print(".",end='',flush=True)
                except:
                    #print(A)
                    #print(B)
                    #print(C[j])
                    #print(D)
                    #print(sdec[i])
                    #print(dec)
                    #print(srad[i])
                    #print(np.sqrt(sx[i]**2.+sy[i]**2.+sz[i]**2.))
                    aaaa = 1
                try:
                    if im(sps[0][0])==0:
                        Cx[-1].insert(j-adjstr,sps[0][0])
                        Cx[-1].append(sps[1][0])
                        Cy[-1].insert(j-adjstr,sps[0][1])
                        Cy[-1].append(sps[1][1])
                    else:
                        adjstr = adjstr + 1
                except:
                    adjstr = adjstr + 1
            else:
                sps = solve([A*(x**2)+B*x+D],x)
                Cx[-1].insert(j,sps[0][0])
                Cx[-1].append(sps[1][0])
                Cy[-1].insert(j,np.sqrt(1.-sps[0][0]**2.))
                Cy[-1].append(-1.*np.sqrt(1.-sps[1][1]**2.))
        if len(Cx[-1])==0:
            del Cx[-1]
            del Cy[-1]
            continue
        Cx[-1] = np.array(Cx[-1])
        Cy[-1] = np.array(Cy[-1])
        Cz.append(sz[i] + srad[i]*Cx[-1])
        Cx[-1] = sx[i] + srad[i]*Cy[-1]*np.cos(np.arange(len(Cz[-1]))*2.*np.pi/len(Cz[-1]))
        Cy[-1] = sy[i] + srad[i]*Cy[-1]*np.sin(np.arange(len(Cz[-1]))*2.*np.pi/len(Cz[-1]))
        try:
            Cr.append(np.sqrt(Cx[-1]**2.+Cy[-1]**2.+Cz[-1]**2.))
        except:
            #print(Cx[-1])
            #print(Cy[-1])
            #print(Cz[-1])
            aaaa = 1
        Cra.append((np.arccos(Cx[-1]/np.sqrt(Cx[-1]**2.+Cy[-1]**2.))*np.sign(Cy[-1])/D2R)%360)
        Cs.append(np.array([Cr[-1]*np.cos(Cra[-1]),Cr[-1]*np.sin(Cra[-1])]).T)
        Cs2.append(Cs[-1])
        Ch.append(ConvexHull(Cs[-1]))
        Chavg.append(np.array([np.sum(Cs[-1].T[0]),np.sum(Cs[-1].T[1])])/len(Cs[-1]))
        Chrad.append(np.amin(np.sum((Cs[-1]-Chavg[-1])**2.,axis=1)))
    if len(Cx)==0:
        return np.array([]),np.array([])
    for i in range(len(Cs2)):
        for j in range(len(Cs)):
            if i==j:
                continue
            cut = np.ones(len(Cs2[i]))
            for k in range(len(Cs2[i])):
                if isin(Cs2[i][k],Cs[j],Ch[j],Chavg[j],Chrad[j]):
                    cut[k] = False
            Cs2[i] = Cs2[i][cut]
    Cp = []
    for i in range(len(Cx)):
        print(Cs2[i].tolist())
        Cp.extend(Cs2[i].tolist())
    Cp = np.array(Cp)
    kdt = cKDTree(Cp)
    Cpi = [0]
    while len(Cpi)<len(Cp):
        if len(Cpi)==1:
            nid = kdt.query(Cp[Cpi[-1]],2)[1][1]
        else:
            nids = kdt.query(Cp[Cpi[-1]],chkdpth+1)[1][1:]
            for k in range(chkdpth):
                if nids[k] not in Cpi[(-1*(chkdpth+1)):-1]:
                    nid = nids[k]
                    break
            nids = kdt.query(Cp[Cpi[-1]],7)[1][1:]
        Cpi.append(nid)
    #Cpi.append(0)
    C1 = np.sqrt(Cp[Cpi].T[0]**2.+Cp[Cpi].T[1]**2.)
    C2 = (np.sign(Cp[Cpi].T[1])*np.arccos(Cp[Cpi].T[0]/C1)+np.pi*(1.-np.sign(Cp[Cpi].T[1])))/D2R
    return C1,C2
    

#convert triangle-slice intersections to ordered boundary
def convint(intr):
    crid = 0
    intr2 = []
    chkln = len(np.unique(intr))
    invtd = False
    while len(intr2)<chkln+1:
        intr2.append(intr[crid])
        if invtd:
            chkloc = crid
            invtd = False
        else:
            chkloc = np.where(intr==intr[crid])[0]
            chkloc = chkloc[chkloc != crid]
        if crid%2==0:
            chkloc2 = np.where(intr==intr[crid+1])[0]
            chkloc2 = chkloc2[chkloc2 != crid+1]
        else:
            chkloc2 = np.where(intr==intr[crid-1])[0]
            chkloc2 = chkloc2[chkloc2 != crid-1]
        try:
            crid = chkloc2[0]
        except:
            crid = 0
            invtd = True
    return intr2

def convint2(intr,intra):
    intx = np.array(intr)*np.cos(np.array(intra)*D2R)
    inty = np.array(intr)*np.sin(np.array(intra)*D2R)
    intX = np.sum(intx)/len(intx)
    intY = np.sum(inty)/len(inty)
    intx2 = intx-intX
    inty2 = inty-intY
    crid = np.argmax(intx2**2.+inty2**2.)
    intsrt = []
    intsrts = [[]]
    chkln = len(np.unique(intr))
    invtd = False
    tcut = np.ones(len(intr),dtype=bool)
    tcut[crid] = False
    tchk = np.arange(len(intr))
    while np.sum([len(jj) for jj in intsrts])<chkln:
        intsrts[-1].append(crid)
        if crid%2==0:
            tcut[crid+1] = False
            chkloc2 = np.where(np.array(intr)[tcut]==intr[crid+1])[0]
            try:
                chkloc2 = tchk[tcut][chkloc2][0]
            except:
                chkloc2 = tchk[tcut][0]
                intsrts.append([])
        else:
            tcut[crid-1] = False
            chkloc2 = np.where(np.array(intr)[tcut]==intr[crid-1])[0]
            try:
                chkloc2 = tchk[tcut][chkloc2][0]
            except:
                chkloc2 = tchk[tcut][0]
                intsrts.append([])
        crid = chkloc2
        #invtd = True
    return [np.array(intr)[intsrts[m]].tolist() for m in range(len(intsrts))],[np.array(intra)[intsrts[m]].tolist() for m in range(len(intsrts))]

def getorder(xs,ys):
    chains = []
    scut = np.zeros(len(xs),dtype=bool)
    for i in range(len(xs)):
        if len(xs[xs==xs[i]])==1:
            scut[i] = True
        elif len(xs[xs==xs[i]])>2:
            print("0",end='',flush=True)
    dists = []
    pairs = []
    for i in range(len(xs)):
        if scut[i]:
            for j in range(i+1,len(xs)):
                if scut[j]:
                    dists.append((xs[i]-xs[j])**2.+(ys[i]-ys[j])**2.)
                    pairs.append([i,j])
    pairs = np.array(pairs)[np.argsort(dists)]
    paird = scut
    xs2 = xs.tolist()
    ys2 = ys.tolist()
    cmp = np.arange(len(xs)).tolist()
    for i in range(len(pairs)):
        if paird[pairs[i][0]] and paird[pairs[i][1]]:
            paird[pairs[i][0]] = False
            paird[pairs[i][1]] = False
            xs2.extend([xs[pairs[i][0]],xs[pairs[i][1]]])
            ys2.extend([ys[pairs[i][0]],ys[pairs[i][1]]])
            cmp.extend([pairs[i][0],pairs[i][1]])
    xs2 = np.array(xs2)
    ys2 = np.array(ys2)
    lcut = np.ones(len(xs2),dtype=bool)
    for i in range(len(xs2)):
        if lcut[i]:
            chains.append([])
            chains[-1].append(cmp[i])
            lcut[i] = False
            j = i + 1 - 2*(i%2)
            while xs2[j] != xs2[i]:
                lcut[j] = False
                k = np.where(xs2==xs2[j])[0]
                k = k[k != j][0]
                chains[-1].append(cmp[k])
                lcut[k] = False
                j = k + 1 - 2*(k%2)
            if chains[-1][0] != chains[-1][-1]:
                chains[-1].append(chains[-1][0])
    return chains

def convint3(intr,intra):
    intx = np.array(intr)*np.cos(np.array(intra)*D2R)
    inty = np.array(intr)*np.sin(np.array(intra)*D2R)
    chkl = []
    ccut = np.ones(len(intr),dtype=bool)
    for i in range(int(len(intr)/2)):
        chkl.append(intx[2*i]+intx[2*i+1])
    chkl = np.array(chkl)
    for i in range(len(chkl)):
        if len(chkl[chkl==chkl[i]])>1:
            ccut[2*i] = False
            ccut[2*i+1] = False
    intx = intx[ccut]
    inty = inty[ccut]
    ocut = getorder(intx,inty)
    icut = np.zeros(len(ocut),dtype=bool)
    lens = np.zeros(len(ocut))
    for i in range(len(ocut)):
        for j in range(len(ocut[i])-1):
            lens[i] = lens[i] + np.sqrt((intx[ocut[i][j+1]]-intx[ocut[i][j]])**2.+(inty[ocut[i][j+1]]-inty[ocut[i][j]])**2.)
    mlh = np.amax(lens)
    for i in range(len(ocut)):
        if lens[i]==mlh:
            continue
        o = ocut[i]
        P = np.array([intx[o][0],inty[o][0]])
        for j in range(len(ocut)):
            if j==i:
                continue
            o1 = ocut[j]
            Ps = np.array([intx[o1],inty[o1]]).T
            if isin2(P,Ps):
                icut[i] = True
                break
    return [[np.array(intr)[ccut][o].tolist(),np.array(intra)[ccut][o].tolist()] for o in ocut],icut 

In [ ]:
def setup_axes3(fig, rect):
    """
    Sometimes, things like axis_direction need to be adjusted.
    """

    # rotate a bit for better orientation
    tr_rotate = Affine2D().translate(-95, 0)

    # scale degree to radians
    tr_scale = Affine2D().scale(np.pi/180., 1.)

    tr = tr_rotate + tr_scale + PolarAxes.PolarTransform()

    grid_locator1 = angle_helper.LocatorDMS(4)
    tick_formatter1 = angle_helper.FormatterDMS()
        
    grid_locator2 = MaxNLocator(3)

    ra0, ra1 = 108, 263
    cz0, cz1 = 0., 306.
    grid_helper = floating_axes.GridHelperCurveLinear(tr,
                                        extremes=(ra0, ra1, cz0, cz1),
                                        grid_locator1=grid_locator1,
                                        grid_locator2=grid_locator2,
                                        tick_formatter1=tick_formatter1,
                                        tick_formatter2=None,
                                        )

    ax1 = floating_axes.FloatingSubplot(fig, rect, grid_helper=grid_helper)
    fig.add_subplot(ax1)

    # adjust axis
    ax1.axis["left"].set_axis_direction("bottom")
    ax1.axis["right"].set_axis_direction("top")

    ax1.axis["bottom"].set_visible(False)
    ax1.axis["top"].set_axis_direction("bottom")
    ax1.axis["top"].toggle(ticklabels=True, label=True)
    ax1.axis["top"].major_ticklabels.set_axis_direction("top")
    ax1.axis["top"].label.set_axis_direction("top")

    ax1.axis["left"].label.set_text(r"r [Mpc h$^{-1}$]")
    ax1.axis["top"].label.set_text(r"$\alpha$")


    # create a parasite axes whose transData in RA, cz
    aux_ax = ax1.get_aux_axes(tr)

    aux_ax.patch = ax1.patch # for aux_ax to have a clip path as in ax
    ax1.patch.zorder=0.8 # but this has a side effect that the patch is
                        # drawn twice, and possibly over some other
                        # artists. So, we decrease the zorder a bit to
                        # prevent this.
    aux_ax.set_facecolor("white")

    return ax1, aux_ax

In [ ]:
#plot VoidFinder maximal spheres
def pvf(dec,wdth,npc):
    #fig = plt.figure(1, figsize=(12,6))
    fig = plt.figure(1, figsize=(1600/96,800/96))
    ax3, aux_ax3 = setup_axes3(fig, 111)
    Cr = cint(dec)
    for i in range(len(vfr)):
        if Cr[i]>0:
            Cr2,Cra2 = gcp(vfr[i],vfra[i],Cr[i],npc)
            aux_ax3.plot(Cra2,Cr2,color='blue')
            aux_ax3.fill(Cra2,Cr2,alpha=0.2,color='blue')
    gdcut = (gr[wflag_vf]*np.sin((gdec[wflag_vf]-dec)*D2R))**2.<wdth**2.
    aux_ax3.scatter(gra[wflag_vf][gdcut],gr[wflag_vf][gdcut],color='k',s=1)
    gdcut = (gr[gflag_vf]*np.sin((gdec[gflag_vf]-dec)*D2R))**2.<wdth**2.
    aux_ax3.scatter(gra[gflag_vf][gdcut],gr[gflag_vf][gdcut],color='red',s=1)
    plt.show()

#plot VoidFinder voids
def pvf2(dec,wdth,npc,chkdpth):
    fig = plt.figure(1, figsize=(1600/96,800/96))
    ax3, aux_ax3 = setup_axes3(fig, 111)
    Cr = cint2(dec)
    for i in range(len(vfr)):
        if np.sum(Cr[i])>0:
            Cr2,Cra2 = gcp2(vfr2[i],vfra2[i],Cr[i],npc,chkdpth)
            aux_ax3.plot(Cra2,Cr2,color='blue')
            aux_ax3.fill(Cra2,Cr2,alpha=0.2,color='blue')
        #Cr2,Cra2 = gcp3(vfx4[i],vfy4[i],vfz4[i],vfr2[i],vfrad2[i],vfdec2[i],dec,npc,chkdpth)
        #if len(Cr2)>0:
        #    aux_ax3.plot(Cra2,Cr2,color='blue')
        #    aux_ax3.fill(Cra2,Cr2,alpha=0.5,color='blue')
    gdcut = (gr[wflag_vf]*np.sin((gdec[wflag_vf]-dec)*D2R))**2.<wdth**2.
    aux_ax3.scatter(gra[wflag_vf][gdcut],gr[wflag_vf][gdcut],color='k',s=1)
    gdcut = (gr[gflag_vf]*np.sin((gdec[gflag_vf]-dec)*D2R))**2.<wdth**2.
    aux_ax3.scatter(gra[gflag_vf][gdcut],gr[gflag_vf][gdcut],color='red',s=1)
    plt.show()

#plot Vsquared voids
def pzbv(dec,wdth):
    fig = plt.figure(1, figsize=(1600/96,800/96))
    ax3, aux_ax3 = setup_axes3(fig, 111)
    Intr,Intra = trint2(dec)
    for i in range(np.amax(trivids)+1):
        if len(Intr[i])>0:
            #Intr2 = convint(Intr[i])
            #Intra2 = convint(Intra[i])
            #Intr2,Intra2 = convint2(Intr[i],Intra[i])
            Intc2,Icut = convint3(Intr[i],Intra[i])
            Intr2 = [Intc[0] for Intc in Intc2]
            Intra2 = [Intc[1] for Intc in Intc2]
            for j in range(len(Intr2)):
                #if Icut[j]:
                #    continue
                aux_ax3.plot(Intra2[j],Intr2[j],color='blue')
                aux_ax3.fill(Intra2[j],Intr2[j],alpha=0.1,color='blue')
            #for j in range(len(Intr2)):
            #    if Icut[j]:
            #        aux_ax3.plot(Intra2[j],Intr2[j],color='blue')
            #        aux_ax3.fill(Intra2[j],Intr2[j],color='white')
    gdcut = (gr[wflag_v2]*np.sin((gdec[wflag_v2]-dec)*D2R))**2.<wdth**2.
    aux_ax3.scatter(gra[wflag_v2][gdcut],gr[wflag_v2][gdcut],color='k',s=1)
    gdcut = (gr[gflag_v2]*np.sin((gdec[gflag_v2]-dec)*D2R))**2.<wdth**2.
    aux_ax3.scatter(gra[gflag_v2][gdcut],gr[gflag_v2][gdcut],color='red',s=1)
    plt.show()

In [ ]:
pzbv(40,5.)

In [ ]:
pvf(40,5.,1000)

In [ ]:
pvf2(40,5.,1000,6)